# Application Package development and testing
## Water Bodies detection

Scenario: Alice implements the Application Package
Note: the full repository on the **OGC EO Application Package Hands-on**, with the three scenarios, is available at: https://github.com/Terradue/ogc-eo-application-package-hands-on.

## Background 
This Application Package takes as input Copernicus Sentinel-2 data and detects water bodies by applying the *Otsu* thresholding technique on the Normalized Difference Water Index (NDWI).

The NDWI is calculated with: 

$$
NDWI = { (green - nir) \over (green + nir) } 
$$

Typically, NDWI values of water bodies are larger than 0.2 and built-up features have positive values between 0 and 0.2. Vegetation has much smaller NDWI values, which results in distinguishing vegetation from water bodies easier. 

The NDWI values correspond to the following ranges:

| Range       | Description                            |
| ----------- | -------------------------------------- |
| 0,2 - 1     | Water surface                          |
| 0.0 - 0,2   | Flooding, humidity                     |
| -0,3 - 0.0  | Moderate drought, non-aqueous surfaces |
| -1 - -0.3   | Drought, non-aqueous surfaces          |

To ease the determination of the water surface/non water surface, the Ostu thresholding technique is used. 

In the simplest form, the Otsu algorithm returns a single intensity threshold that separate pixels into two classes, foreground and background. This threshold is determined by minimizing intra-class intensity variance, or equivalently, by maximizing inter-class variance:

![image](https://upload.wikimedia.org/wikipedia/commons/3/34/Otsu%27s_Method_Visualization.gif)

## Application Workflow
The Water Bodies detection steps are depicted below:
``` mermaid
graph TB
  A[STAC Items] --> B
  A[STAC Items] --> C
subgraph Process STAC item
  B["crop(green)"] --> D[Normalized difference];
  C["crop(nir)"] --> D[Normalized difference];
  D --> E[Otsu threshold]
end
  E --> F[Create STAC]
```

The application takes a list of Sentinel-2 STAC items references and then crops the radiometric bands `green` and `NIR` with a user-defined area of interest (AOI). Each cropped band is then used to calculate the `NDWI` and subsequently the Otsu threashold is applied to it, generating the water bodies output mask. The final step of the workflow consists on generating the STAC catalog and items for the generated results.

Alice organizes the Application Package to include a macro workflow that reads the list of Sentinel-2 STAC items references, the AOI and the EPSG code. The workflow steps include i) a sub-workflow for the detection of the water bodies and ii) a step to create the STAC catalog of the generated output product(s)

![image](FOSS4Gworkshop_07AppPackage/docs/water_bodies.png "water-bodies")

The sub-workflow applies the  `crop`, `Normalized difference`, `Otsu threshold` steps:

![image](FOSS4Gworkshop_07AppPackage/docs/detect_water_body.png "detect-water-body")

## Input Sentinel-2 acquisitions
The development and test dataset is made of two Sentinel-2 acquisitions:

| Acquisitions 	|Image 1                    	|Image 2                    	|
|--------------	|---------------------------	|---------------------------	|
| Date         	|2021-07-13                 	|2022-05-24                 	|
| URL          	| [S2B_10TFK_20210713_0_L2A](https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A) 	| [S2A_10TFK_20220524_0_L2A](https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A) 	|
| Quicklook    	| ![image](FOSS4Gworkshop_07AppPackage/docs/img_20210713.jpg) 	| ![image](FOSS4Gworkshop_07AppPackage/docs/img_20220504.jpg) 	|

## Environments creation

Each `Command Line Tool` step (`crop`, `Normalized difference`, `Otsu threshold` and `Create STAC`) runs a Python script in a dedicated environment / container. 
To generate the environments, open a new `Terminal` and execute the commands below (either one by one or all at once).

**Note**: This configuration step takes around five minutes to complete.

In [1]:
# Node "CROP" environment
!mamba create -c conda-forge -y -p /srv/conda/envs/env_crop gdal click pystac 

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        Supported by @QuantStack

        GitHub:  https://github.com/QuantStack/mamba
        Twitter: https://twitter.com/QuantStack

████████████████████████████████████████

In [2]:
# Node "NORMALISED_DIFFERENCE" environment
!mamba create -c conda-forge -y -p /srv/conda/envs/env_norm_diff click gdal  

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        Supported by @QuantStack

        GitHub:  https://github.com/QuantStack/mamba
        Twitter: https://twitter.com/QuantStack

████████████████████████████████████████

In [3]:
# Node "OTSU" environment
!mamba create -c conda-forge -y -p /srv/conda/envs/env_otsu gdal scikit-image click 

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        Supported by @QuantStack

        GitHub:  https://github.com/QuantStack/mamba
        Twitter: https://twitter.com/QuantStack

████████████████████████████████████████

In [4]:
# Node "STAC" environment
!mamba create -c conda-forge -y -p /srv/conda/envs/env_stac click pystac python=3.9 pip && \
    /srv/conda/envs/env_stac/bin/pip install rio_stac

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        Supported by @QuantStack

        GitHub:  https://github.com/QuantStack/mamba
        Twitter: https://twitter.com/QuantStack

████████████████████████████████████████

In [5]:
# Clean unnecessary packages
!mamba clean --all -f -y

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        Supported by @QuantStack

        GitHub:  https://github.com/QuantStack/mamba
        Twitter: https://twitter.com/QuantStack

████████████████████████████████████████

## Dockers creation

In [7]:
import os
os.chdir('FOSS4Gworkshop_07AppPackage')

In [8]:
!docker build -t simonevaccari/crop:0.1.0 --no-cache -f water-bodies/command-line-tools/crop/Dockerfile .

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
WARN[0000] "/" is not a shared mount, this could cause issues or missing mounts with rootless containers 
STEP 1/13: FROM docker.io/mambaorg/micromamba:latest
Trying to pull docker.io/mambaorg/micromamba:latest...
Getting image source signatures
Copying blob 9e3ea8720c6d [==>--------------------------------] 2.7MiB / 29.9MiB
Copying blob 4f4fb700ef54 [======================================] 32.0b / 32.0b
Copying blob a7aee41143c3 [====================================] 283.0b / 283.0b
Copying blob ee8f42978de7 [====================================] 522.0b / 522.0b
Copying blob abcf63bffa78 [===>------------------------------] 620.9KiB / 5.4MiB
Copying blob bdd19a82bedd [================================] 112.5KiB / 112.5KiB
Copying blob 9e3ea8720c6d [=======>---------------------------] 6.6MiB / 29.9MiB
Copying blob 4f4fb700ef54 done  
Copying blob a7aee41143c3 done  
Copying blob ee8f42978de7 done  
Copying bl

In [9]:
!docker build -t simonevaccari/norm_diff:0.1.0 --no-cache -f water-bodies/command-line-tools/norm_diff/Dockerfile .

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
STEP 1/13: FROM docker.io/mambaorg/micromamba:latest
STEP 2/13: USER root
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> ffb422349d8
STEP 3/13: ENV USERNAME=mambauser 
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> 238f9e75d1c
STEP 4/13: ENV PATH=/srv/conda/envs/env_norm_diff/bin:$PATH
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> c39ab02236e
STEP 5/13: RUN apt-get update -y  && apt-get upgrade -y  && apt-get install -y --no-install-recommends         ca-certificates         curl sudo git gcc build-essential
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security 

In [10]:
!docker build -t simonevaccari/otsu:0.1.0 --no-cache -f water-bodies/command-line-tools/otsu/Dockerfile .

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
STEP 1/14: FROM docker.io/mambaorg/micromamba:latest
STEP 2/14: USER root
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> 2cbc9b9464a
STEP 3/14: ENV USERNAME=mambauser 
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> e52e4f2a4d5
STEP 4/14: ENV PATH=/srv/conda/envs/env_otsu/bin:$PATH
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> a440dde9c69
STEP 5/14: RUN apt-get update -y  && apt-get upgrade -y  && apt-get install -y --no-install-recommends         ca-certificates         curl sudo git gcc build-essential
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRel

In [11]:
!docker build -t simonevaccari/stac:0.1.0 --no-cache -f water-bodies/command-line-tools/stac/Dockerfile .

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
STEP 1/13: FROM docker.io/mambaorg/micromamba:latest
STEP 2/13: USER root
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> 239a96eafb9
STEP 3/13: ENV USERNAME=mambauser 
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> 377cd221245
STEP 4/13: ENV PATH=/srv/conda/envs/env_stac/bin:$PATH
WARN[0000] SHELL is not supported for OCI image format, [/usr/local/bin/_dockerfile_shell.sh] will be ignored. Must use `docker` format 
--> a59f57e2d64
STEP 5/13: RUN apt-get update -y  && apt-get upgrade -y  && apt-get install -y --no-install-recommends         ca-certificates         curl sudo git gcc build-essential
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRel

## Application Package inspection

Open the `app-package.cwl` Application Package and familiarise yourself with its structure, to understand what's going on during execution:  

1. Inspect the main workflow which `id` is **`water_bodies`**: 
    1.1. What are the input parameters? *(stac_items, aoi, epsg)*
    1.2. What are the steps of this workflow? *(node_water_bodies, node_stac)* 
2. Inspect the workflow which `id` is **`detect_water_body`**:
    2.1. What are the steps of this workflow? *(node_crop, node_normalized_difference, node_otsu)*
3. Inspect each of the `CommandLineTool` of `id`: **`crop`**, **`norm_diff`**, **`otsu`** and **`stac`** 
    3.1. Inspect each of the `Dockerfile` 


In [19]:
print('Dockers built successfully:')
!docker images

Dockers built successfully:
/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
REPOSITORY                         TAG         IMAGE ID      CREATED         SIZE
localhost/simonevaccari/stac       0.1.0       31d309350812  4 minutes ago   1.24 GB
localhost/simonevaccari/otsu       0.1.0       9ee6e0707783  6 minutes ago   2.44 GB
localhost/simonevaccari/norm_diff  0.1.0       3f4f8923266c  11 minutes ago  2.08 GB
localhost/simonevaccari/crop       0.1.0       1fa483aa5638  14 minutes ago  2.08 GB
docker.io/mambaorg/micromamba      latest      bcfdda4930c6  2 weeks ago     98.2 MB


## Application Package execution

The water bodies Application Package can be executed with: 
```
cwltool --no-container app-package.cwl#water_bodies params.yml > out.json
```
where:
* `cwltool` is a Common Workflow Language runner. 
* The flag `--no-container` is used to instruct `cwltool` to use the local command-line tools instead of using the containers.
* `app-package.cwl#water_bodies` defines the CWL file to execute as well as the entry point after the `#` symbol. Here it's the `Workflow` with the id `water_bodies`.
* The file **`params.yml`** is used to define the input parameters. In this case, these are:

```
stac_items:
- "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A"
- "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"

aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"
```
* `out.json` is used to store the execution logs 

In [14]:
# Execute the app-package with the local modules without using containers
!cwltool --no-container water-bodies/app-package.cwl#water_bodies water-bodies/params.yml > out.json

In [ ]:
# Execute the app-package with containers
!cwltool water-bodies/app-package.cwl#water_bodies water-bodies/params.yml > out.json

## Result inspection

The execution of the `cwltool` generates the output `out.json` file, as well as a folder which name is a 8-character alphanumeric string. In this folder are stored the generated `catalog.json` and the `otsu.tif` and related STAC item for each of the two input Sentinel-2 images, in the structure below:
* `catalog.json`
* `S2A_10TFK_20220524_0_L2A`
    * `otsu.tif`
    * `S2A_10TFK_20220524_0_L2A.json`
* `S2A_10TFK_20220524_0_L2A`
    * `otsu.tif`
    * `S2A_10TFK_20220524_0_L2A`

You can plot the output `otsu.tif` files with the `visualisation.ipynb` Jupyter Notebook. This Notebook uses `pystac` to access the geotiffs produced, `leafmap` to plot the tiles served by a local tile server. Open the Notebook and run all cells. 

First, run the commands below to create the environment:

In [ ]:
# "VISUALISATION" environment
!mamba create -c conda-forge -y -q -p /srv/conda/envs/env_visual pystac ipykernel jupyterlab localtileserver jupyter-server-proxy pip && \
    /srv/conda/envs/env_visual/bin/pip install leafmap

In [ ]:
# Import liraries
from localtileserver import TileClient, get_leaflet_tile_layer
import leafmap.foliumap as leafmap

import pystac
import json
import numpy as np
import os

# Define env variables for localtileserver
os.environ["GTIFF_SRS_SOURCE"] = "EPSG"
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = 'proxy/{port}'
os.environ['PROJ_DATA'] = '/srv/conda/envs/env_visual/share/proj/'
os.environ['GDAL_DATA'] = '/srv/conda/envs/env_visual/share/gdal/'

**Note** if you get a `ModuleNotFoundError: No module named 'localtileserver'` error: click on **Kernel** on the top tab, click on **Change Kernel...** and select `Python [conda env:env_visual]`. Then re-run the cell above.

Load the JSON result listing generated by `cwltool`:

In [ ]:
with open("out.json") as f: 
    results = json.load(f)

Look for the `catalog.json` file:

In [ ]:
for item in results["stac_catalog"]["listing"]:
    
    if item['basename'] == "catalog.json":
        catalog = pystac.read_file(item["path"])
        break

List the contents of the STAC Catalog

In [ ]:
catalog.describe()

In [ ]:
it1 = catalog.get_item('S2B_10TFK_20210713_0_L2A')
it1

In [ ]:
it1.properties

In [ ]:
m = leafmap.Map()

for item in catalog.get_all_items():
    m.add_raster(item.get_assets()["data"].get_absolute_href(), layer_name=item.id) #, crs="EPSG:32610")

m